In [1]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
from scipy.stats import norm
import statsmodels.api as sm

/Users/bhchen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/bhchen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# Q1
## 1)Present a table of summary statistics for the key variables including the borrower’ age, gender, loan amount, interest rate, credit scores, a dummy whether the borrower has a frequent contact, approval dummy, and delinquency dummy

In [2]:
path = '/Users/bhchen/Desktop/FIN3210/Week 1 Assignment/FIN3210 Week 1 Data.csv'  
df = pd.read_csv(path, encoding='gbk')
usedList = ['age', 'gender','instalments_num','instalments_amount','deal', 'default',
            'nominalrates','creditlevelasbuyer', 'tencentscore','gaodescore',
            'numbercontact20s','highcontact20s']
df = df[usedList]

In [3]:
# choose the most valid credit score among 3 scores
df_score = df[['creditlevelasbuyer', 'tencentscore','gaodescore']]
df_score.describe()

,creditlevelasbuyer,tencentscore,gaodescore
count,4031.000000,5000.000000,5000.000000
mean,53.119077,58.608168,0.201975
std,108.629757,14.218112,0.076724
min,0.000000,9.000000,0.023518
25%,0.000000,53.888889,0.192094
50%,14.000000,60.200000,0.192094
75%,58.000000,65.258929,0.192094
max,1830.000000,98.000000,0.732120


In [4]:
df_contact = df[['highcontact20s', 'numbercontact20s']]
df_0 = df_contact[df.highcontact20s == False]
df_1 = df_contact[df.highcontact20s == True]
print(df_0.describe())
print(df_1.describe())

       numbercontact20s
count       2489.000000
mean           0.349538
std            0.675091
min            0.000000
25%            0.000000
50%            0.000000
75%            0.000000
max            2.000000
       numbercontact20s
count       2511.000000
mean          13.776583
std           23.501205
min            3.000000
25%            5.000000
50%            8.000000
75%           15.000000
max          751.000000


In [5]:
dummyList = ['gender', 'default','highcontact20s']
for item in dummyList:
    df[item] = df[item].map({True:1, False:0})

# Amount of loan principal in thousands of Chinese Yuan
df['amount'] = df['instalments_amount'] / 1000

# take the logarithm value of loan amount, and credit score
log_name = ['amount','age','tencentscore']
for variable in log_name:
    log_name = f'log_{variable}'
    df[log_name] = np.log(df[variable])

In [6]:
variableList =['age', 'gender','amount', 'log_amount', 'nominalrates', 'tencentscore', 'highcontact20s', 'deal', 'default']
q1_result = df[variableList].describe()
q1_result

,age,gender,amount,log_amount,nominalrates,tencentscore,highcontact20s,deal,default
count,5000.000000,5000.000000,5000.00000,5000.000000,4997.000000,5000.000000,5000.000000,5000.000000,2205.000000
mean,27.675400,0.146600,406.20142,5.948887,0.276058,58.608168,0.502200,0.441400,0.419501
std,8.326146,0.353742,130.62336,0.356948,0.085912,14.218112,0.500045,0.496604,0.493589
min,18.000000,0.000000,50.00000,3.912023,0.130080,9.000000,0.000000,0.000000,0.000000
25%,21.000000,0.000000,320.00000,5.768321,0.204560,53.888889,0.000000,0.000000,0.000000
50%,25.000000,0.000000,398.00000,5.986452,0.204579,60.200000,1.000000,0.000000,0.000000
75%,32.000000,0.000000,498.00000,6.210600,0.359347,65.258929,1.000000,1.000000,1.000000
max,56.000000,1.000000,869.00000,6.767343,0.494185,98.000000,1.000000,1.000000,1.000000


# Q2
## 2)Perform a logit regression and examine the relation between the delinquency likelihood and credit scores

In [7]:
# check the correlation of 6 variables
model_var = ['age', 'gender', 'log_amount', 'nominalrates', 'log_tencentscore', 'highcontact20s']
correlation_matrix = df[model_var].corr()
correlation_matrix

,age,gender,log_amount,nominalrates,log_tencentscore,highcontact20s
age,1.000000,0.031102,-0.111748,0.026481,0.037632,0.063281
gender,0.031102,1.000000,0.013164,0.012485,0.014044,0.007789
log_amount,-0.111748,0.013164,1.000000,-0.002569,-0.049765,0.027558
nominalrates,0.026481,0.012485,-0.002569,1.000000,-0.018111,0.012270
log_tencentscore,0.037632,0.014044,-0.049765,-0.018111,1.000000,-0.045698
highcontact20s,0.063281,0.007789,0.027558,0.012270,-0.045698,1.000000


In [8]:
df_reg = df[model_var + ['default', 'deal']]
# Fill missing values in 'nominalrates' with its median
median_rate = df_reg['nominalrates'].median()
df_reg['nominalrates'].fillna(median_rate, inplace=True)

/var/folders/74/xjyr_wpj7njfggsgt60kbgvc0000gn/T/ipykernel_63438/1276176932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg['nominalrates'].fillna(median_rate, inplace=True)


In [9]:
df_reg1 = df_reg.dropna(subset=['default'])

y1 = df_reg1['default']
X1 = df_reg1[model_var]
X1 = sm.add_constant(X1)

logit_model_default = sm.Logit(y1, X1)
result_default = logit_model_default.fit()
result_default.summary()

Optimization terminated successfully.
         Current function value: 0.672331
         Iterations 5


/Users/bhchen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                 2205
Model:                          Logit   Df Residuals:                     2198
Method:                           MLE   Df Model:                            6
Date:                Tue, 19 Sep 2023   Pseudo R-squ.:                 0.01147
Time:                        17:29:01   Log-Likelihood:                -1482.5
converged:                       True   LL-Null:                       -1499.7
Covariance Type:            nonrobust   LLR p-value:                 5.643e-06
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -3.3429      0.980     -3.412      0.001      -5.263      -1.423
age                 -0.0119      0.006     -2.077      0.038      -0.023      -0.001
gender               0.0093      0.114      0.082      0.935      -0.214       0.233
log_amount           0.1945      0.127      1.533      0.125      -0.054       0.443
nominalrates         1.4255      0.538      2.648      0.008       0.370       2.480
log_tencentscore     0.4098      0.129      3.177      0.001       0.157       0.663
highcontact20s       0.2895      0.087      3.311      0.001       0.118       0.461
====================================================================================
"""

In [10]:
coefficients = result_default.params
t_values = result_default.tvalues

# Creating a DataFrame to display the results
results1 = pd.DataFrame({
    'Coefficient': coefficients,
    't-statistic': t_values
})
results1

,Coefficient,t-statistic
const,-3.342945,-3.412056
age,-0.011921,-2.076960
gender,0.009344,0.081976
log_amount,0.194474,1.533472
nominalrates,1.425457,2.648189
log_tencentscore,0.409841,3.176869
highcontact20s,0.289512,3.310638


# Q3
## 3)Perform a logit regression and examine the relation between the loan approval likelihood and credit scores

In [11]:
y2 = df_reg['deal']
X2 = df_reg[model_var]
X2 = sm.add_constant(X2)

logit_model_deal = sm.Logit(y2, X2)
result_deal = logit_model_deal.fit()
result_deal.summary()

Optimization terminated successfully.
         Current function value: 0.641622
         Iterations 5


/Users/bhchen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   deal   No. Observations:                 5000
Model:                          Logit   Df Residuals:                     4993
Method:                           MLE   Df Model:                            6
Date:                Tue, 19 Sep 2023   Pseudo R-squ.:                 0.06505
Time:                        17:29:01   Log-Likelihood:                -3208.1
converged:                       True   LL-Null:                       -3431.3
Covariance Type:            nonrobust   LLR p-value:                 2.906e-93
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                9.7843      0.726     13.474      0.000       8.361      11.208
age                 -0.0179      0.004     -4.896      0.000      -0.025      -0.011
gender               0.5021      0.084      5.992      0.000       0.338       0.666
log_amount          -0.6377      0.085     -7.480      0.000      -0.805      -0.471
nominalrates         3.2145      0.347      9.266      0.000       2.535       3.894
log_tencentscore    -1.6766      0.111    -15.048      0.000      -1.895      -1.458
highcontact20s       0.1301      0.060      2.176      0.030       0.013       0.247
====================================================================================
"""

In [12]:
coefficients2 = result_deal.params
t_values2 = result_deal.tvalues

# Creating a DataFrame to display the results
results2 = pd.DataFrame({
    'Coefficient': coefficients2,
    't-statistic': t_values2
})
results2

,Coefficient,t-statistic
const,9.784330,13.474101
age,-0.017945,-4.896224
gender,0.502097,5.992126
log_amount,-0.637720,-7.479910
nominalrates,3.214482,9.265516
log_tencentscore,-1.676634,-15.047906
highcontact20s,0.130107,2.175874


# Q4
## 4)Perform a logit regression and examine the relation between the loan approval likelihood and the dummy whether the borrower has a frequent contact

In [13]:
result_deal.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   deal   No. Observations:                 5000
Model:                          Logit   Df Residuals:                     4993
Method:                           MLE   Df Model:                            6
Date:                Tue, 19 Sep 2023   Pseudo R-squ.:                 0.06505
Time:                        17:29:01   Log-Likelihood:                -3208.1
converged:                       True   LL-Null:                       -3431.3
Covariance Type:            nonrobust   LLR p-value:                 2.906e-93
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                9.7843      0.726     13.474      0.000       8.361      11.208
age                 -0.0179      0.004     -4.896      0.000      -0.025      -0.011
gender               0.5021      0.084      5.992      0.000       0.338       0.666
log_amount          -0.6377      0.085     -7.480      0.000      -0.805      -0.471
nominalrates         3.2145      0.347      9.266      0.000       2.535       3.894
log_tencentscore    -1.6766      0.111    -15.048      0.000      -1.895      -1.458
highcontact20s       0.1301      0.060      2.176      0.030       0.013       0.247
====================================================================================
"""

In [14]:
results2

,Coefficient,t-statistic
const,9.784330,13.474101
age,-0.017945,-4.896224
gender,0.502097,5.992126
log_amount,-0.637720,-7.479910
nominalrates,3.214482,9.265516
log_tencentscore,-1.676634,-15.047906
highcontact20s,0.130107,2.175874
